In [1]:
from dotenv import load_dotenv
import logging
import pandas as pd
from sources import earwolf, rss

load_dotenv()
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%d-%b-%y %H:%M:%S')

In [ ]:
# Scrape data from sources and write to file

earwolf.scrape().to_csv("data/earwolf_episodes.csv", index=False)
rss.scrape().to_csv("data/rss_episodes.csv", index=False)

In [ ]:
# Read separate episode sources and merge

earwolf_eps = pd.read_csv("data/earwolf_episodes.csv")
rss_eps = pd.read_csv("data/rss_episodes.csv")

episodes = pd.merge(earwolf_eps, rss_eps, on=[
                    "number", "bestOf", "live"], how="inner")
episodes.to_csv("data/merged_episodes.csv", index=False)

In [5]:
episodes = pd.read_csv("data/merged_episodes.csv")
episodes.head(10)

,number,bestOf,title,live,releaseDate,guests,earwolfUrl
0,692.0,False,"Chalke Talk, Structure-A-Stuffy & Tunt",False,2021-02-08,"['Sarah Chalke', 'Paul F. Tompkins', 'Katie Ri...",https://www.earwolf.com/episode/chalke-talk-st...
1,691.0,False,#NoStank,False,2021-02-01,"['Tim Baltz', 'Shaun Diston', 'Lily Sullivan']",https://www.earwolf.com/episode/nostank/
2,690.0,False,Spices On Your Meat,False,2021-01-25,"['Mary Lynn Rajskub', 'Ego Nwodim', 'Carl Tart...",https://www.earwolf.com/episode/spices-on-your...
3,689.0,False,Marketing Mine Own Selfowitz,False,2021-01-18,"['Jake Fogelnest', 'Dan Lippert', 'Lyric Lewis']",https://www.earwolf.com/episode/marketing-mine...
4,688.0,False,The Sugarhill Gang Museum,False,2021-01-11,"['Edi Patterson', 'Paul F. Tompkins']",https://www.earwolf.com/episode/the-sugarhill-...
5,687.0,False,Full Grown Grogu,False,2021-01-04,"['Ben Schwartz', 'Horatio Sanz']",https://www.earwolf.com/episode/full-grown-grogu/
6,2020.4,True,Best of 2020 Part 4,False,2020-12-31,['Paul F. Tompkins'],https://www.earwolf.com/episode/best-of-2020-p...
7,2020.3,True,Best of 2020 Part 3,False,2020-12-28,['Paul F. Tompkins'],https://www.earwolf.com/episode/best-of-2020-p...
8,2020.2,True,Best of 2020 Part 2,False,2020-12-24,['Paul F. Tompkins'],https://www.earwolf.com/episode/best-of-2020-p...
9,2020.1,True,Best of 2020 Part 1,False,2020-12-21,['Paul F. Tompkins'],https://www.earwolf.com/episode/best-of-2020-p...
